<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Making_the_Most_of_your_Colab_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making the Most of your Colab Subscription



## Faster GPUs

Users who have purchased one of Colab's paid plans have access to faster GPUs and more memory. You can upgrade your notebook's GPU settings in `Runtime > Change runtime type` in the menu to select from several accelerator options, subject to availability.

The free of charge version of Colab grants access to Nvidia's T4 GPUs subject to quota restrictions and availability.

You can see what GPU you've been assigned at any time by executing the following cell. If the execution result of running the code cell below is "Not connected to a GPU", you can change the runtime by going to `Runtime > Change runtime type` in the menu to enable a GPU accelerator, and then re-execute the code cell.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In order to use a GPU with your notebook, select the `Runtime > Change runtime type` menu, and then set the hardware accelerator to the desired option.

## More memory

Users who have purchased one of Colab's paid plans have access to high-memory VMs when they are available. More powerful GPUs are always offered with high-memory VMs.



You can see how much memory you have available at any time by running the following code cell. If the execution result of running the code cell below is "Not using a high-RAM runtime", then you can enable a high-RAM runtime via `Runtime > Change runtime type` in the menu. Then select High-RAM in the Runtime shape toggle button. After, re-execute the code cell.


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Longer runtimes

All Colab runtimes are reset after some period of time (which is faster if the runtime isn't executing code). Colab Pro and Pro+ users have access to longer runtimes than those who use Colab free of charge.

## Background execution

Colab Pro+ users have access to background execution, where notebooks will continue executing even after you've closed a browser tab. This is always enabled in Pro+ runtimes as long as you have compute units available.



## Relaxing resource limits in Colab Pro

Your resources are not unlimited in Colab. To make the most of Colab, avoid using resources when you don't need them. For example, only use a GPU when required and close Colab tabs when finished.



If you encounter limitations, you can relax those limitations by purchasing more compute units via Pay As You Go. Anyone can purchase compute units via [Pay As You Go](https://colab.research.google.com/signup); no subscription is required.

## Send us feedback!

If you have any feedback for us, please let us know. The best way to send feedback is by using the Help > 'Send feedback...' menu. If you encounter usage limits in Colab Pro consider subscribing to Pro+.

If you encounter errors or other issues with billing (payments) for Colab Pro, Pro+, or Pay As You Go, please email [colab-billing@google.com](mailto:colab-billing@google.com).

## More Resources

### Working with Notebooks in Colab
- [Overview of Colab](/notebooks/basic_features_overview.ipynb)
- [Guide to Markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)

<a name="working-with-data"></a>
### Working with Data
- [Loading data: Drive, Sheets, and Google Cloud Storage](/notebooks/io.ipynb)
- [Charts: visualizing data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

### Machine Learning Crash Course
These are a few of the notebooks from Google's online Machine Learning course. See the [full course website](https://developers.google.com/machine-learning/crash-course/) for more.
- [Intro to Pandas DataFrame](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
- [Linear regression with tf.keras using synthetic data](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/linear_regression_with_synthetic_data.ipynb)


<a name="using-accelerated-hardware"></a>
### Using Accelerated Hardware
- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TensorFlow with TPUs](/notebooks/tpu.ipynb)

<a name="machine-learning-examples"></a>

## Machine Learning Examples

To see end-to-end examples of the interactive machine learning analyses that Colab makes possible, check out these tutorials using models from [TensorFlow Hub](https://tfhub.dev).

A few featured examples:

- [Retraining an Image Classifier](https://tensorflow.org/hub/tutorials/tf2_image_retraining): Build a Keras model on top of a pre-trained image classifier to distinguish flowers.
- [Text Classification](https://tensorflow.org/hub/tutorials/tf2_text_classification): Classify IMDB movie reviews as either *positive* or *negative*.
- [Style Transfer](https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization): Use deep learning to transfer style between images.
- [Multilingual Universal Sentence Encoder Q&A](https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa): Use a machine learning model to answer questions from the SQuAD dataset.
- [Video Interpolation](https://tensorflow.org/hub/tutorials/tweening_conv3d): Predict what happened in a video between the first and the last frame.


In [ ]:
from torch import nn
from transformers import Wav2Vec2Model, AutoModelForAudioClassification
from transformers import AutoFeatureExtractor
from datasets import Dataset, DatasetDict
import torch
import torch.nn.functional as F


def init_weight(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
    if isinstance(m, torch.nn.BatchNorm2d):
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
    elif isinstance(m, torch.nn.Conv2d):
        m.weight.data.normal_(0.0, 0.02)


class Projection(torch.nn.Module):

    def __init__(self, in_planes, out_planes=None, n_layers=1, layer_type=0):
        super(Projection, self).__init__()
        self.out_planes = out_planes
        if out_planes is None:
            out_planes = in_planes
        self.layers = torch.nn.Sequential()
        _in = None
        _out = None
        for i in range(n_layers):
            # self.layers.add_module(f"{i}fc",
            #                        torch.nn.Linear(_in, _out))
            self.layers.add_module(f"{i}cv1d",
                                   torch.nn.Conv1d(in_planes, in_planes, 3, padding='same'))
            self.layers.add_module(f"{i}bn",
                                    torch.nn.BatchNorm1d(in_planes))

            if layer_type == n_layers - 1:
                self.layers.add_module(f"{i}relu",
                                            torch.nn.LeakyReLU(.2))
            # if i < n_layers - 1:
            #     # if layer_type > 0:
            #     #     self.layers.add_module(f"{i}bn",
            #     #                            torch.nn.BatchNorm1d(_out))
            #     if layer_type > 1:
            #         self.layers.add_module(f"{i}relu",
            #                                torch.nn.LeakyReLU(.2))
        self.apply(init_weight)

    def forward(self, x):
        # x = .1 * self.layers(x) + x
        x = self.layers(x)
        # x = x.reshape(x.shape[0],-1)
        # x = F.adaptive_avg_pool1d(x, self.out_planes)
        x = x.mean(dim=1)
        return x

class Discriminator(torch.nn.Module):
    def __init__(self, in_planes, n_layers=3, hidden=None):
        super(Discriminator, self).__init__()

        _hidden = in_planes if hidden is None else hidden
        self.body = torch.nn.Sequential()
        for i in range(n_layers-1):
            _in = in_planes if i == 0 else _hidden
            _hidden = int(_hidden // 1.5) if hidden is None else hidden
            self.body.add_module('block%d'%(i+1),
                                 torch.nn.Sequential(
                                    #  nn.Dropout(0.2),
                                     torch.nn.Linear(_in, _hidden),
                                     torch.nn.BatchNorm1d(_hidden),
                                     torch.nn.LeakyReLU(0.2)
                                 ))
        self.tail = torch.nn.Linear(_hidden, 1)
        self.apply(init_weight)

    def forward(self,x):
        x = self.body(x)
        x = self.tail(x)
        return x

class Discriminator_Conv(torch.nn.Module):
    def __init__(self, in_planes, n_layers=3, hidden=None):
        super(Discriminator, self).__init__()

        _hidden = in_planes if hidden is None else hidden
        self.body = torch.nn.Sequential()
        for i in range(n_layers-1):
            _in = in_planes if i == 0 else _hidden
            _hidden = int(_hidden // 1.5) if hidden is None else hidden
            self.body.add_module('block%d'%(i+1),
                                 torch.nn.Sequential(
                                    #  nn.Dropout(0.2),
                                     torch.nn.Linear(_in, _hidden),
                                     torch.nn.BatchNorm1d(_hidden),
                                     torch.nn.LeakyReLU(0.2)
                                 ))
        self.tail = torch.nn.Linear(_hidden, 1, bias=False)
        self.apply(init_weight)

    def forward(self,x):
        x = self.body(x)
        x = self.tail(x)
        return x

class Wave_Network(nn.Module):
    def __init__(self, num_classes=12, model_path='', device='cuda'):
        super().__init__()
        self.device = device
        self.backbone = Wav2Vec2Model.from_pretrained(model_path).to(device=device)

        for param in self.backbone.parameters():
            param.requires_grad = False
        self.backbone.feature_extractor._freeze_parameters()

        self.discriminator = Discriminator(768).to(device=device)
        self.projection = Projection(768, 768, 2).to(device=device)
        self.projector = nn.Sequential(
            nn.Dropout(0.3),
            nn.LazyLinear(256).to(device=device)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            # nn.Flatten(),
            # nn.LazyLinear(256).to(device=device),
            nn.LazyLinear(num_classes).to(device=device)
        )

    def get_train_params(self, is_classification=True):
        if is_classification:
            return [
                {'params': self.projector.parameters()},
                {'params': self.classifier.parameters()},
            ]
        else:
            return [
                {'params': self.projection.parameters()},
                {'params': self.discriminator.parameters()},
            ]

    def forward(self, x, nomaly_label=None, std=0.05, is_train=True):
        x = self.backbone(x).last_hidden_state
        x_reshape = x.reshape(x.shape[0],-1)
        x_hidden= F.adaptive_avg_pool1d(x_reshape, 768)
        x_hidden_state = self.projector(x)
        x_hidden_state = x_hidden_state.mean(dim=1)

        if is_train:
            normal_hidden_state = x_hidden_state[nomaly_label==1]
            x_classification = self.classifier(normal_hidden_state)

            x_projector = self.projection(x_hidden)
            # add noise
            # if is_train:
            normal_x_projector = x_projector[nomaly_label==1]
            noise = torch.normal(mean=0, std=std, size=normal_x_projector.shape).to(device=self.device)
            x_noise = normal_x_projector + noise
            x_noise = torch.concat((x_projector, x_noise))
            # else:
            #     x_noise = x_projector
            x_anomaly = self.discriminator(x_noise)
        else:
            x_classification = self.classifier(x_hidden_state)
            x_projector = self.projection(x_hidden)
            x_anomaly = self.discriminator(x_projector)
        return x_anomaly, x_classification

    def save(self, file_path):
        # Custom logic before saving
        print(f"Saving model to {file_path}")

        # For example, saving both the model's state dict and some metadata
        torch.save({
            'model_state_dict_backbone': self.backbone.state_dict(),
            'model_state_dict_discriminator': self.discriminator.state_dict(),
            'model_state_dict_projection': self.projection.state_dict(),
            'model_state_dict_classifier': self.classifier.state_dict(),
            'custom_metadata': {
                'info': 'This is a custom saved model',
                'epoch': 10,
                'loss': 0.1234
            }
        }, file_path)

        # Custom logic after saving
        print("Model saved successfully!")

    def load(self, file_path):
        # Custom logic before saving
        print(f"Loading model from {file_path}")

        # For example, saving both the model's state dict and some metadata
        temp_model = torch.load(file_path)
        self.backbone.load_state_dict(temp_model['model_state_dict_backbone'])
        self.discriminator.load_state_dict(temp_model['model_state_dict_discriminator'])
        self.projection.load_state_dict(temp_model['model_state_dict_projection'])
        self.classifier.load_state_dict(temp_model['model_state_dict_classifier'])
        # Custom logic after saving
        print("Model loaded successfully!")

class Wave_Network_Classification(nn.Module):
    def __init__(self, num_classes=12):
        super().__init__()

        self.projector = nn.Sequential(
            # nn.Dropout(0.3),
            # nn.Conv1d(49, 1, 3, padding='same'),
            nn.LazyLinear(256)
        )

        self.squeeze_exhibition_in = nn.LazyLinear(16)
        self.relu = nn.ReLU()
        self.squeeze_exhibition_out = nn.LazyLinear(49)
        self.sigmoid = nn.Sigmoid()

        self.classifier = nn.Sequential(
            # nn.Dropout(0.3),
            nn.LazyLinear(num_classes)
        )

    def forward(self, x, nomaly_label=None, is_train=True):
        input_x = x[nomaly_label==1]
        x_hidden_state = self.projector(input_x) #[n, 49, 256]
        # x_hidden_state = torch.squeeze(x_hidden_state)
        # x_hidden_state_mean = x_hidden_state.mean(dim=1)
        x_hidden_state_mean = x_hidden_state.mean(dim=-1) #[n, 49, 1]
        x_hidden_state_mean = torch.unsqueeze(x_hidden_state_mean, 1)
        # x_hidden_state_transpose = torch.transpose(x_hidden_state_mean, -1, -2) # [n, 1, 49]
        x_squeeze_exhibition = self.squeeze_exhibition_in(x_hidden_state_mean)
        x_squeeze_exhibition = self.relu(x_squeeze_exhibition)
        x_squeeze_exhibition = self.squeeze_exhibition_out(x_squeeze_exhibition)
        x_squeeze_exhibition = self.sigmoid(x_squeeze_exhibition)

        x_hidden_state = torch.transpose(x_hidden_state, -1, -2)
        x_hidden_state = torch.mul(x_hidden_state, x_squeeze_exhibition)
        x_hidden_state = torch.transpose(x_hidden_state, -1, -2)

        x_hidden_state = x_hidden_state.mean(dim=1)

        if is_train:
            normal_hidden_state = x_hidden_state
            x_classification = self.classifier(normal_hidden_state)
        else:
            x_classification = self.classifier(x_hidden_state)
        return x_classification

class Wave_Network_Anomaly_Detection(nn.Module):
    def __init__(self, std=0.05):
        super().__init__()
        self.discriminator = Discriminator(768)
        self.projection = Projection(49, 768, 5)
        self.std = std

    def forward(self, x, nomaly_label=None, is_train=True):
        # x_reshape = x.reshape(x.shape[0],-1)
        # x_hidden= F.adaptive_avg_pool1d(x_reshape, 768)
        # x_hidden = x.mean(dim=1)
        x_hidden = x
        if is_train:
            x_projector = self.projection(x_hidden)
            # add noise
            # if is_train:
            normal_x_projector = x_projector[nomaly_label==1]
            noise = torch.normal(mean=0, std=self.std, size=normal_x_projector.shape).cuda()
            x_noise = normal_x_projector + noise
            x_noise = torch.concat((x_projector, x_noise))
            # else:
            #     x_noise = x_projector
            x_anomaly = self.discriminator(x_noise)
        else:
            x_projector = self.projection(x_hidden)
            x_anomaly = self.discriminator(x_projector)
        return x_anomaly
# _network =  Wav2Vec2Model.from_pretrained(pretrained_model_name_or_path='./model_1')
# print(_network)
# for name, param in _network.named_parameters():
#     print(f"layer: {name} | Shape: {param.shape}")

# model = AutoModelForAudioClassification.from_pretrained(
#         pretrained_model_name_or_path='./model_1', num_labels=10, ignore_mismatched_sizes=True)
# for name, param in model.named_parameters():
#     print(f"layer: {name} | Shape: {param.shape}")